<a href="https://colab.research.google.com/github/swarthyPig/Raspberry-Pi_DHT22_Weather_Station_with_correlation-and-ML/blob/master/machine%20learning/merge_dataFrame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from pandas import DataFrame
import numpy as np


th = pd.read_json("http://203.241.246.114:3030/th")
cr = pd.read_json("http://203.241.246.114:3030/cr_value/cr")

iot_th = th[['date', 'temperature', 'humidity']]
iot_cr = cr[['date', 'cr']]


In [0]:
'''
ths와 crs가 담긴 테이터 프레임을 date를 기준으로 outer방식으로 병합한다 
'''
iot_data = pd.merge(iot_th, iot_cr, on='date', how="outer") 
#iot_data.set_index('date',inplace=True) # date를 열로 설정한다

iot_data = iot_data.sort_values(['date'], ascending=[True]) # date를 기준으로 오름차순 정렬

# 온도 습도 값 계산을 위해 형변환
iot_data['temperature'] = pd.to_numeric(iot_data['temperature'])
iot_data['humidity'] = pd.to_numeric(iot_data['humidity'])
iot_data['cr'] = pd.to_numeric(iot_data['cr'])


idx = list(np.where(iot_data['cr'] > 0 )[0])  # cr값이 0보다 큰값에 대한 인덱스 반환


length = len(idx)
for i in range(0,length):
    
    idx_num = idx[i]
    
    # 인덱스를 -1,1로 하면 연속으로 NaN이 있는 부분에서 에러발생. 3초마다 나오는 온도 습도 값의 차이가 거의 없으므로 -1,-2 인덱스로 계산
    iot_data['temperature'].iloc[idx_num] = round((iot_data['temperature'].iloc[idx_num-1] + iot_data['temperature'].iloc[idx_num-2])/2,2)
    iot_data['humidity'].iloc[idx_num] = round((iot_data['humidity'].iloc[idx_num-1] + iot_data['humidity'].iloc[idx_num-2])/2,2)

iot_data = iot_data.iloc[idx] # 전처리가 완료된 데이터 프레임으로 변경

del iot_data['date']

iot_data.to_csv("iot_data.csv", mode='w') # export csv

iot_data






/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,temperature,humidity,cr
379627,21.40,52.20,4.0
379628,22.55,50.60,5.0
379629,22.50,50.50,4.0
379630,21.90,53.45,5.0
379631,21.90,53.50,5.0
379632,21.90,53.60,5.0
379633,21.90,53.10,5.0
379634,21.90,53.25,5.0
379635,21.90,53.25,5.0
379636,21.90,53.52,5.0


In [0]:
iot_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 598 entries, 379627 to 380224
Data columns (total 3 columns):
temperature    598 non-null float64
humidity       598 non-null float64
cr             598 non-null float64
dtypes: float64(3)
memory usage: 18.7 KB
